In [ ]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
from os import path
sns.set()
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
import datetime as dt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import calendar
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline
import time
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.layers import Conv1D, Flatten,MaxPooling1D
from keras.preprocessing import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import auc
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string, nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

ENGLISH_STOPWORDS = stopwords.words('english')
PUNCTUATIONS = string.punctuation
LEMMATIZER = WordNetLemmatizer()
STEMMER = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Project-NLP/train-balanced-sarcasm-50k.csv')
df.head()

,label,comment
0,1,Yes that's why I don't advocate for the rights...
1,1,I can find you a Pinsir if you want one that bad.
2,0,"Huh, that's pretty interesting actually thanks..."
3,1,TIL a monsoon = average summer weather in Sweden
4,1,Yeah everyone knows holographic tigers and bom...


In [ ]:
df.dropna(subset=['comment'], inplace=True)
def clean(tweet):
    #emojis
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    tweet =  emoji_pattern.sub(r'', tweet)

    # usernames mentions like "@abc123"
    ment = re.compile(r"(@[A-Za-z0-9]+)")
    tweet =  ment.sub(r'', tweet)

    # Character entity references
    tweet = re.sub(r"&amp;", "&", tweet)

    # html tags
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    tweet = re.sub(html, '', tweet)

    # Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
    tweet = re.sub(r'https?://\S+|www\.\S+','', tweet)

    #Punctuations and special characters
    tweet = re.sub('[%s]' % re.escape(PUNCTUATIONS),'',tweet)

    tweet = tweet.lower()

    splits = tweet.split()
    splits = [word for word in splits if word not in set(ENGLISH_STOPWORDS)]
    tweet = ' '.join(splits)

    return tweet

In [ ]:
df['cleaned_text'] = df['comment'].apply((lambda x: clean(x)))
print("Cleaned")

Cleaned


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   label         49997 non-null  int64 
 1   comment       49997 non-null  object
 2   cleaned_text  49997 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.5+ MB


In [ ]:
vocab_size = 1000
maxlen = 1000
embedding_dims = 50
filters = 32
kernel_size = 3
hidden_dims = 250

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['label'], test_size=0.2, random_state=42)

tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
cnn_model = Sequential()
cnn_model.add(Embedding(vocab_size,embedding_dims,input_length=maxlen))
cnn_model.add(Dropout(.5))
cnn_model.add(Conv1D(filters, kernel_size,padding = 'valid', activation = 'relu'))
cnn_model.add(MaxPooling1D())
cnn_model.add(Conv1D(filters, kernel_size,padding = 'valid', activation = 'relu'))
cnn_model.add(MaxPooling1D())
cnn_model.add(Flatten())
cnn_model.add(Dense(hidden_dims, activation ='relu'))
cnn_model.add(Dropout(.5))
cnn_model.add(Dense(1, activation='sigmoid'))


cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


cnn_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=64)

Epoch 1/30
625/625 [==============================] - 26s 31ms/step - loss: 0.6850 - accuracy: 0.5315 - val_loss: 0.6631 - val_accuracy: 0.5865
Epoch 2/30
625/625 [==============================] - 7s 12ms/step - loss: 0.6564 - accuracy: 0.5943 - val_loss: 0.6586 - val_accuracy: 0.5911
Epoch 3/30
625/625 [==============================] - 6s 10ms/step - loss: 0.6470 - accuracy: 0.6045 - val_loss: 0.6607 - val_accuracy: 0.5874
Epoch 4/30
625/625 [==============================] - 6s 10ms/step - loss: 0.6411 - accuracy: 0.6097 - val_loss: 0.6634 - val_accuracy: 0.5880
Epoch 5/30
625/625 [==============================] - 6s 9ms/step - loss: 0.6359 - accuracy: 0.6130 - val_loss: 0.6663 - val_accuracy: 0.5840
Epoch 6/30
625/625 [==============================] - 5s 9ms/step - loss: 0.6309 - accuracy: 0.6179 - val_loss: 0.6697 - val_accuracy: 0.5834
Epoch 7/30
625/625 [==============================] - 6s 10ms/step - loss: 0.6245 - accuracy: 0.6213 - val_loss: 0.6736 - val_accuracy: 0.5809


In [ ]:
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test, batch_size=64)
print("The test loss is :",test_loss)
print("\nThe test Accuracy is :",test_accuracy*100)

157/157 [==============================] - 1s 5ms/step - loss: 0.7443 - accuracy: 0.5780
The test loss is : 0.7442865371704102

The test Accuracy is : 57.8000009059906


In [ ]:
predictions = cnn_model.predict(X_test)
treshold = 0.5
pred = (predictions > treshold).astype(int)
f1 = f1_score(y_test, pred)
print(f"F1-Score {f1}")

313/313 [==============================] - 1s 3ms/step
F1-Score 0.46730623579904074
